In [1]:
import numpy as np
import pandas as pd

In [5]:
players = pd.read_csv('../dataset light memory/players_fifa22.csv')
teams = pd.read_csv('../dataset light memory/teams_fifa22.csv')

In [ ]:
list(players.columns)

In [6]:
import random
from collections import deque
import os
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
        
class DQNetwork:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.learning_rate = 0.01
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim = self.state_size, activation = 'relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'linear'))
        model.compile(loss='mse', optimizer = Adam(learning_rate = self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in batch:
            target = reward #target = 80
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0])) 
            target_cur = self.model.predict(state)
            target_cur[0][action] = target
            self.model.fit(state, target_cur, epochs = 1, verbose = 0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    def load(self, name):
        self.model.load_weights(name)
    def save(self, name):
        self.model.save_weights(name)
    


In [4]:
import gym

env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1001

In [6]:
arr = env.reset()

In [55]:
done = False
env.reset()
while not done:
    a, b , done, _ = env.step(0)
    print(a,b,done)

[-0.00625125 -0.22040966  0.03027815  0.29822242] 1.0 False
[-0.01065945 -0.41594985  0.0362426   0.60029864] 1.0 False
[-0.01897844 -0.61155957  0.04824857  0.90417355] 1.0 False
[-0.03120963 -0.8073006   0.06633205  1.2116231 ] 1.0 False
[-0.04735565 -1.0032132   0.0905645   1.5243329 ] 1.0 False
[-0.06741991 -1.1993045   0.12105117  1.8438541 ] 1.0 False
[-0.091406   -1.3955357   0.15792824  2.171549  ] 1.0 False
[-0.11931671 -1.5918063   0.20135923  2.5085294 ] 1.0 False
[-0.15115283 -1.7879367   0.2515298   2.8555813 ] 1.0 True


In [ ]:
# agent = DQNetwork(state_size, action_size)

# for e in range(n_episodes):
#     state = env.reset()
#     state = np.reshape(state, [1, state_size])
#     for time in range(5000):
#         #env.render()
#         action = agent.act(state)
#         next_state, reward, done, _ = env.step(action)
#         reward = reward if not done else -10
#         next_state = np.reshape(next_state, [1, state_size])
#         agent.remember(state, action, reward, next_state, done)
#         state = next_state
#         if done:
#             print(f"episode: {e}/{n_episodes}, score: {time}, e: {round(agent.epsilon, 2)}")
#             break
#     if len(agent.memory) > batch_size:
#         agent.replay(batch_size)

In [ ]:
state = env.reset()
state = np.reshape(state, [1, state_size])
next_state, reward, done, _ = env.step(action)

In [ ]:
testdqn.model.predict([[1,2,3,4,5,6,7,8,9,10]])

## env logic

#### env reset()
#### env.step()
#### env.

In [7]:
class Player:
    def __init__(self, name = '', position = '', position_rating = 0, value = 0, wage = 0):
        self.name = name
        #self.overall_rating = overall_rating
        self.position = position
        self.position_rating = position_rating
        self.value = value
        #self.wage = wage

In [68]:
class Environment:
    def __init__(self, players):
        self.players = players[['ID', 'Name', 'ValueEUR', 'Overall', 'STRating', 'LWRating', 'LFRating', 'CFRating', 'RFRating', 'RWRating',
                                'CAMRating', 'LMRating', 'CMRating', 'RMRating', 'LWBRating',
                                'CDMRating', 'RWBRating', 'LBRating', 'CBRating', 'RBRating', 'GKRating']]
        self.formation = ['GK', 
                         'LB', 'CB', 'CB', 'RB', 
                         'LM', 'CAM', 'RM',
                         'LW', 'ST', 'RW']
        self.reward = 0
    def reset(self):
        self.money = np.random.randint(10000000, 1000000000, size = 1)[0]
        self.formation = ['GK', 
                         'LB', 'CB', 'CB', 'RB', 
                         'LM', 'CAM', 'RM',
                         'LW', 'ST', 'RW']
        self.posid = 0
        self.done = False
        self.squad = []
        self.squad_players_ids = [-1 for i in range (11)]
        state = [self.money, self.posid, *self.squad_players_ids]
        return state
    
    def step(self, action):
        money_exceeded_flag = False
        if action in self.squad_players_ids:
            return [[self.money, self.posid, *self.squad_players_ids], -10, False, {}]
        
        if len(self.squad) + 1 == 11:
            self.done = True
        self.squad_players_ids[self.posid] = action
        player = Player(players.iloc[action]['Name'], 
                                 self.formation[self.posid],
                                 players.iloc[action][f'{self.formation[self.posid]}Rating'],
                                 players.iloc[action]['ValueEUR'])
        self.squad.append(player)
        self.posid += 1
        self.money -= player.value
        if self.money < 0:
            money_exceeded_flag = True
        next_state = [self.money, self.posid, *self.squad_players_ids]
        
        reward = self.evalSquad()
        if money_exceeded_flag == True:
            done = True
            reward = -100
            return [next_state, reward, self.done, {}]
        return [next_state, reward, self.done, {}]
    
    def evalSquad(self):
        sum_rating = 0
        for player in self.squad:
            sum_rating += player.position_rating
        if len(self.squad) == 11:
            return sum_rating
        return round(sum_rating/len(self.squad), 2)

In [67]:
players.shape[0]

19260

In [77]:
env = Environment(players)
state_size = 13
action_size = players.shape[0]
batch_size = 32
n_episodes = 200

agent = DQNetwork(state_size, action_size)

for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(5000):
        #env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"episode: {e}/{n_episodes}, score: {reward}, e: {round(agent.epsilon, 2)}")
            break
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

episode: 0/200, score: 520, e: 1.0
episode: 1/200, score: 710, e: 1.0
episode: 2/200, score: 654, e: 1.0
episode: 3/200, score: 610, e: 0.99
episode: 4/200, score: 572, e: 0.99
episode: 5/200, score: 536, e: 0.99
episode: 6/200, score: 656, e: 0.98
episode: 7/200, score: 515, e: 0.98
episode: 8/200, score: 574, e: 0.97
episode: 9/200, score: 604, e: 0.97
episode: 10/200, score: 517, e: 0.96
episode: 11/200, score: 662, e: 0.96
episode: 12/200, score: 550, e: 0.95
episode: 13/200, score: 617, e: 0.95
episode: 14/200, score: 583, e: 0.94
episode: 15/200, score: 478, e: 0.94
episode: 16/200, score: 599, e: 0.93
episode: 17/200, score: 594, e: 0.93
episode: 18/200, score: 571, e: 0.92
episode: 19/200, score: 582, e: 0.92
episode: 20/200, score: 613, e: 0.91
episode: 21/200, score: 632, e: 0.91
episode: 22/200, score: 607, e: 0.9
episode: 23/200, score: 582, e: 0.9
episode: 24/200, score: 668, e: 0.9
episode: 25/200, score: 546, e: 0.89
episode: 26/200, score: 648, e: 0.89
episode: 27/200, 

In [69]:
env = Environment(players)

In [70]:
env.reset()

[729833271, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

In [71]:
env.step(9999)

[[729208271, 1, 9999, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], 19.0, False, {}]

In [ ]:
for player in env.squad:
    print(f'name: {player.name}, position: {player.position}, Position rating: {player.position_rating}')

In [ ]:
print(env.evalSquad())